In [73]:
# Instalação do Pyspark 
!pip install pyspark 


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [74]:
# Importação Bibliotecas / Finções


import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import * 

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable      #Resolver problema de versão Python x Pyspark Workers 

In [75]:
# Criando e Iniciando Sessão PySpark 
spark = (
    SparkSession.builder     #Cria a Sessão 
    .master('local')         # Cluster  
    .appName('PySpark_01')   # Nome da Sessão 
    .getOrCreate()           # Cria a Sessão ou reiniciar uma existente 
)

In [76]:
# Link do DataSet:  https://www.kaggle.com/datasets/djamshed/fifa-world-cup-2018-players/discussion?sort=hotness
# Cria DataFrame e Ler Arquivos 
df = spark.read.csv('C:/Users/rodrigo.cerqueira/Documents/Spark/wc2018-players.csv', header=True, inferSchema=True)  #Header verifica se contem cabeçalho e o infer verifica os Types das colunas e criar o dataframe  

In [77]:
df.show() 

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
|Argentina|  4|  DF|  ANSALDI Cristian|20.09.1986|   ANSALDI|     Torino FC (ITA)|   181|    73|
|Argentina|  5|  MF|      BIGLIA Lucas|30.01.1986|    BIGLIA|      AC Milan (ITA)|   175|    73|
|Argentina|  7|  MF|       BAN

In [78]:
df.printSchema()   # Semelhante ao df.info do Pandas. 
                   # Resultado do inferSchema 

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



In [79]:
df.toPandas().isna().sum()  # Transformando para DF Pandas, verincando valore nulos e somandos. (OBS: Por ser um DataSet pequeno usamos o Pandas )

Team                 0
#                    0
Pos.                 0
FIFA Popular Name    0
Birth Date           0
Shirt Name           0
Club                 0
Height               0
Weight               0
dtype: int64

In [80]:
for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count()) 

# 1 - Obs: vai dar um erro na "Coluna Pos."   (devido ao ponto), 
# 2   vamos corrigie na proxima linha. 

Team 0
# 0


AnalysisException: Syntax error in attribute name: Pos..

In [81]:
# Renomear Colunas 

df = df.withColumnRenamed('Pos.', 'Posicao')\
.withColumnRenamed('Team', 'Selecao')\
.withColumnRenamed ('#',  'Numero')\
.withColumnRenamed ('FIFA Popular Name', 'Nome_FIFA')\
.withColumnRenamed ('Birth Date', 'Nascimento')\
.withColumnRenamed ('Shirt Name', 'Nome Camiseta')\
.withColumnRenamed ('Club', 'Time')\
.withColumnRenamed ('Height', 'Altura')\
.withColumnRenamed ('Weight', 'Peso')
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [82]:
df.columns    # Lista colunas 

['Selecao',
 'Numero',
 'Posicao',
 'Nome_FIFA',
 'Nascimento',
 'Nome Camiseta',
 'Time',
 'Altura',
 'Peso']

In [83]:
# Vai verificar linha por linhas os valores Nulos e contar 
for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count()) 

Selecao 0
Numero 0
Posicao 0
Nome_FIFA 0
Nascimento 0
Nome Camiseta 0
Time 0
Altura 0
Peso 0


In [84]:
# Selecionar Colunas
df.select('Selecao', 'Nome_FIFA').show(5)


+---------+------------------+
|  Selecao|         Nome_FIFA|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



In [85]:
# Outra Forma de Fazer chamando com a função (COL)  
df.select(col('Selecao'), col('Nome_FIFA')).show(5)

+---------+------------------+
|  Selecao|         Nome_FIFA|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



In [86]:
# Outra Forma de Fazer chamando com a função (COL)
df.select(df['Selecao'])


DataFrame[Selecao: string]

In [87]:
#Selecionar Colunas com ALIAS (Apelidos)
df.select(col('Selecao').alias('Time')).show(5)

+---------+
|     Time|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



In [88]:
#Organizar SELECT 
df.select('Nome_Fifa', 'Peso', 'Altura').show(5)

+------------------+----+------+
|         Nome_Fifa|Peso|Altura|
+------------------+----+------+
|TAGLIAFICO Nicolas|  65|   169|
|    PAVON Cristian|  65|   169|
|    LANZINI Manuel|  66|   167|
|    SALVIO Eduardo|  69|   167|
|      MESSI Lionel|  72|   170|
+------------------+----+------+
only showing top 5 rows



In [89]:
# Filtrar DF
df.filter('Selecao = "Brazil"').show(10)

+-------+------+-------+-----------------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|        Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+-----------------+----------+-------------+--------------------+------+----+
| Brazil|    18|     MF|             FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|           TAISON|13.01.1988|       TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|      FERNANDINHO|04.05.1985|  FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|           FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    10|     FW|           NEYMAR|05.02.1992|    NEYMAR JR|Paris Saint-Germa...|   175|  68|
| Brazil|    11|     MF|PHILIPPE COUTINHO|12.06.1992|  P. COUTINHO|  FC Barcelona (ESP)|   172|  68|
| Brazil|     7|     FW|    DOUGLAS COSTA|14.09.1990|     D. COSTA|   Juventus FC (ITA)|   

In [90]:
df.filter(col('Nome Camiseta') == "FRED").show()  # Quando tiver espaço no nome da coluna é preciso usar a função (COL)

+-------+------+-------+---------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+---------+----------+-------------+--------------------+------+----+
| Brazil|    18|     MF|     FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
+-------+------+-------+---------+----------+-------------+--------------------+------+----+



In [91]:
#Filtrar DF com 2 Condições (AND / &)

df.filter((col('Selecao') == "Argentina") & (col('Altura') >= 180)).show()


+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     4|     DF|  ANSALDI Cristian|20.09.1986|      ANSALDI|     Torino FC (ITA)|   181|  73|
|Argentina|     9|     FW|   HIGUAIN Gonzalo|10.12.1987|      HIGUAÍN|   Juventus FC (ITA)|   184|  75|
|Argentina|    13|     MF|  MEZA Maximiliano|15.12.1992|         MEZA|CA Independiente ...|   180|  76|
|Argentina|    23|     GK|CABALLERO Wilfredo|28.09.1981|    CABALLERO|    Chelsea FC (ENG)|   186|  80|
|Argentina|     2|     DF|   MERCADO Gabriel|18.03.1987|      MERCADO|    Sevilla FC (ESP)|   181|  81|
|Argentina|    17|     DF|  OTAMENDI Nicolas|12.02.1988|     OTAMENDI|Manchester City F...|   181|  81|
|Argentina|    16|     DF|       ROJO Marcos|20.03.1990|        

In [92]:
#Filtrar DF com 2 Condições ( OR / |)   (True True)

df.filter((col('Nome_FIFA') == "MESSI Lionel") | (col('Nome_FIFA') == "SALVIO Eduardo")).show(5)
                                                    


+---------+------+-------+--------------+----------+-------------+------------------+------+----+
|  Selecao|Numero|Posicao|     Nome_FIFA|Nascimento|Nome Camiseta|              Time|Altura|Peso|
+---------+------+-------+--------------+----------+-------------+------------------+------+----+
|Argentina|    18|     DF|SALVIO Eduardo|13.07.1990|       SALVIO|  SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|  MESSI Lionel|24.06.1987|        MESSI|FC Barcelona (ESP)|   170|  72|
+---------+------+-------+--------------+----------+-------------+------------------+------+----+



In [93]:
#Filtrar DF Combinado & e | (AND e OR)

df.filter((col('Selecao') == "Brazil") & (col('Posicao') == 'DF') |(col('Nome_FIFA') == "MESSI Lionel") | (col('Nome_FIFA') == "SALVIO Eduardo")).show()

+---------+------+-------+--------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|     Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+--------------+----------+-------------+--------------------+------+----+
|Argentina|    18|     DF|SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|  MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
|   Brazil|    22|     DF|        FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
|   Brazil|     6|     DF|   FILIPE LUIS|09.08.1985|  FILIPE LUIS|Atletico Madrid (...|   182|  73|
|   Brazil|    13|     DF|    MARQUINHOS|14.05.1994|   MARQUINHOS|Paris Saint-Germa...|   183|  75|
|   Brazil|     3|     DF|       MIRANDA|07.09.1984|      MIRANDA|FC Internazionale...|   186|  78|
|   Brazil|    14|     DF|        DANILO|15.07.1991|       DANILO|Manchester City F...|   184|  78|


In [94]:
# Criar Novas Colunas (Função LIT)

df.withColumn('WorldCup', lit(2018)).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|WorldCup|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    2018|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|    2018|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|    2018|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|    2018|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|    2018|
+---------+------+-------+------------------+----------+-------------+--------------------+-----

In [95]:
df_convertido = df.withColumn("Altura_m", round(col("Altura") / 100,2))  # Criei uma coluna com Altura em Float
df_convertido.show()


+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Altura_m|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    1.69|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|    1.69|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|    1.67|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|    1.67|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|     1.7|
|Argentina|     4|     DF|  ANSALDI Cristian|20.09.1986|      ANSALDI|     Torino FC (ITA)|   18

In [96]:
#Criar Coluna Condicional (Usando Função Subtring)  
df.withColumn('Sub', substring('Selecao', 1, 3)).show(5)


+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Sub|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|Arg|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|Arg|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|Arg|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|Arg|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|Arg|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+
only showing top 5 rows



In [97]:
df = df.withColumn('Ano', substring('Nascimento', -4, 4))     # -4 indica os quantro ultimos digitos, no caso da coluna Nascimento o ANO. 


In [98]:
# Criar Coluna Condicional (Concat / concat_ws)   (Com separador e sem )
df.withColumn('Concat', concat('Selecao', col('Nome Camiseta'))).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+-------------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|             Concat|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+-------------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|ArgentinaTAGLIAFICO|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|     ArgentinaPAVÓN|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|   ArgentinaLANZINI|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|    ArgentinaSALVIO|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170

In [99]:
# Criar Coluna Condicional (Concat / concat_ws)   (Com separador e sem )

df.withColumn('Separador', concat_ws(' - ', 'Selecao', 'Numero', 'Posicao')).show(5)


+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+-------------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|          Separador|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+-------------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992| Argentina - 3 - DF|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|Argentina - 22 - MF|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|Argentina - 15 - MF|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|Argentina - 18 - DF|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170

In [100]:
df.show()

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|1987|
|Argentina|     4|     DF|  ANSALDI Cristian|20.09.1986|      ANSALDI|     Torino FC (ITA)|   181|  73|1986|
|Argentina|     5| 

In [101]:
# Mudar o Tipo da coluna  
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: string (nullable = true)



In [102]:
 # Transsformando a Coluna Ano em um Inteiro 
df = df.withColumn('Ano', col('Ano').cast(IntegerType()))  

In [103]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: integer (nullable = true)



In [104]:
# Corrigir Data de nascimento de texto para DATA. 





In [105]:
# Criando uma coluna Dia.
dia = udf(lambda data: data.split('.')[0])

In [106]:
df = df.withColumn('Dia', dia('Nascimento'))

In [107]:
df.show()

Py4JJavaError: An error occurred while calling o550.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 118.0 failed 1 times, most recent failure: Lost task 0.0 in stage 118.0 (TID 96) (DELEST4-DTF50F.educacao.intragov executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.Interruptib

In [108]:
mes = udf(lambda data: data.split('.')[1])

In [109]:
df = df.withColumn('Mes', mes('Nascimento'))

In [110]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Dia: string (nullable = true)
 |-- Mes: string (nullable = true)



In [111]:
df = df.withColumn('Data_Nascimento', concat_ws('.', 'Ano', 'Mes', 'Dia'))

In [112]:
df.show()

Py4JJavaError: An error occurred while calling o577.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 119.0 failed 1 times, most recent failure: Lost task 0.0 in stage 119.0 (TID 97) (DELEST4-DTF50F.educacao.intragov executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.Interruptib

In [114]:
df.withColumn('Data_Nascimento', col('Data_Nascimento').cast(DateType())).show()

Py4JJavaError: An error occurred while calling o589.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 120.0 failed 1 times, most recent failure: Lost task 0.0 in stage 120.0 (TID 98) (DELEST4-DTF50F.educacao.intragov executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.Interruptib